In [298]:
import pandas as pd
import numpy as np

In [342]:
interval = 'M'
market_cap = pd.read_csv('../data/market-cap.csv')
market_cap = market_cap.sort_index(level=0, axis=1)
market_cap = market_cap[['mkvalt', 'tic', 'datadate']]
market_cap = market_cap.drop_duplicates(['tic', 'datadate']).set_index(['tic', 'datadate'])
market_cap = market_cap.unstack(level=0).swaplevel(0, 1, axis=1).sort_index(axis=1).sort_index(axis=0)
market_cap.columns = market_cap.columns.droplevel(1)
market_cap.index = pd.to_datetime(market_cap.index)

In [343]:
pattern = r'^[a-zA-Z]+$'
mask = market_cap.columns.get_level_values(0).str.match(pattern)
mask = pd.Series(mask).fillna(False).to_numpy()
market_cap = market_cap.loc[:, mask]
market_cap = market_cap.sort_index()
market_cap = market_cap.fillna(method='ffill')
market_cap = market_cap.sort_index(axis=0)

In [351]:
def get_top_n(row, n=1000):
    return row.sort_values(ascending=False, na_position='last')[:n].index.tolist()

In [378]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)
market_cap_copy = market_cap.copy()
top_n_per_year = market_cap_copy.parallel_apply(get_top_n, axis=1)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [379]:
interval_to_save = 'D'
top_n_resampled = top_n_per_year
top_n_resampled.index = pd.to_datetime(top_n_resampled.index)
top_n_resampled = top_n_resampled.resample(interval_to_save, convention='end').ffill()
top_n_resampled

datadate
2002-01-31    [WMT, HD, DELL, TGT, LOW, KSS, BDRBF, KR, GPS,...
2002-02-01    [WMT, HD, DELL, TGT, LOW, KSS, BDRBF, KR, GPS,...
2002-02-02    [WMT, HD, DELL, TGT, LOW, KSS, BDRBF, KR, GPS,...
2002-02-03    [WMT, HD, DELL, TGT, LOW, KSS, BDRBF, KR, GPS,...
2002-02-04    [WMT, HD, DELL, TGT, LOW, KSS, BDRBF, KR, GPS,...
                                    ...                        
2020-06-26    [MSFT, AAPL, GOOGL, AMZN, FB, JPM, NVGI, JNJ, ...
2020-06-27    [MSFT, AAPL, GOOGL, AMZN, FB, JPM, NVGI, JNJ, ...
2020-06-28    [MSFT, AAPL, GOOGL, AMZN, FB, JPM, NVGI, JNJ, ...
2020-06-29    [MSFT, AAPL, GOOGL, AMZN, FB, JPM, NVGI, JNJ, ...
2020-06-30    [MSFT, AAPL, GOOGL, AMZN, FB, JPM, NVGI, JNJ, ...
Freq: D, Length: 6726, dtype: object

In [374]:
import json

top_n_str_dates = top_n_resampled
top_n_str_dates.index = top_n_resampled.index.strftime('%Y-%m-%d')
top_n_dict = top_n_str_dates.to_dict()
with open(f'../data/yearly_top_1000_mktcap_{interval_to_save}.json', 'w') as outfile:
    json.dump(top_n_dict, outfile)